In [152]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
import os
import json
from pathlib import Path
import re

In [153]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

#### Flat Data File for SystemOps

In [103]:
query = "SELECT distinct i.tableId, i.rowIndex, p.pdfName, p.date, p.company, p.monitoring_year, p.monitoring_year_ordinal, pc.consultantName, pr.application_title, i.pipelineName, t.tableTitle, i.land_use_standardized, i.status, i.status_confidence, i.vec_pri, i.vec_sec, w.vec, i.issue_pri, i.issue_sec, s.sub_vec FROM issues i LEFT JOIN tables t ON i.tableId = t.headTable LEFT JOIN pdfs p ON t.pdfName = p.pdfName LEFT JOIN projects pr ON p.application_id = pr.application_id LEFT JOIN pdfsconsultants pc ON p.pdfName = pc.pdfName LEFT JOIN sub_vecs s ON i.tableId = s.tableId AND i.rowIndex = s.rowIndex LEFT JOIN word2vec w ON i.tableId = w.tableId AND i.rowIndex = w.rowIndex WHERE i.status IS NOT NULL;"

with engine.connect() as conn:
    issues_df = pd.read_sql(query, conn)
issues_df.fillna('', inplace = True)
issues_df = issues_df.astype(str)
#issues_df[['monitoring_year', 'monitoring_year_ordinal', 'date', 'status_confidence']] = issues_df[['monitoring_year', 'monitoring_year_ordinal', 'date', 'status_confidence']].astype(str)
issues_df = issues_df.groupby(['tableId', 'rowIndex'], as_index = False, sort = False).agg(lambda x: ', '.join(set(x))) # https://stackoverflow.com/questions/50357837/aggregating-string-columns-using-pandas-groupby

In [105]:
query = "SELECT distinct ip.tableId, ip.rowIndex, ip.rowCounter, p.pdfName, p.date, p.company, p.monitoring_year, p.monitoring_year_ordinal, pc.consultantName, pr.application_title, i.pipelineName, t.tableTitle, i.land_use_standardized, ip.status_issue_parsed, ip.status_confidence_issue_parsed, ip.issue_parsed, w.vec, s.sub_vec FROM pcmr.issues_parsed ip LEFT JOIN sub_vecs s ON ip.tableId = s.tableId AND ip.rowIndex = s.rowIndex AND ip.rowCounter = s.rowCounter LEFT JOIN word2vec w ON ip.tableId = w.tableId AND ip.rowIndex = w.rowIndex AND ip.rowCounter = w.rowCounter LEFT JOIN issues i ON ip.tableId = i.tableId AND ip.rowIndex = i.rowIndex LEFT JOIN tables t ON i.tableId = t.headTable LEFT JOIN pdfs p ON t.pdfName = p.pdfName LEFT JOIN projects pr ON p.application_id = pr.application_id LEFT JOIN pdfsconsultants pc ON p.pdfName = pc.pdfName;"
with engine.connect() as conn:
    issues_parsed_df = pd.read_sql(query, conn)
issues_parsed_df.fillna('', inplace = True)
issues_parsed_df = issues_parsed_df.astype(str)
issues_parsed_df = issues_parsed_df.groupby(['tableId', 'rowIndex', 'rowCounter'], as_index = False, sort = False).agg(lambda x: ', '.join(set(x))) # https://stackoverflow.com/questions/50357837/aggregating-string-columns-using-pandas-groupby

In [109]:
issues_parsed_df.rename({'status_issue_parsed': 'status', 'status_confidence_issue_parsed': 'status_confidence'}, axis = 1, inplace = True)

In [133]:
result = issues_parsed_df.append(issues_df, ignore_index = True, sort = False)
result.to_csv('pcmr_environmental_issues.csv', encoding = 'utf-8-sig')

### Rough Work

In [85]:
query = "SELECT distinct ip.tableId, ip.rowIndex, ip.rowCounter, p.pdfName, p.date, p.company, p.monitoring_year, p.monitoring_year_ordinal, pc.consultantName, pr.application_title, i.pipelineName, t.tableTitle, i.land_use_standardized, ip.status_issue_parsed, ip.status_confidence_issue_parsed, ip.issue_parsed, w.vec, s.sub_vec FROM issues_parsed ip LEFT JOIN issues i ON ip.tableId = i.tableId AND ip.rowIndex = i.rowIndex LEFT JOIN tables t ON i.tableId = t.headTable LEFT JOIN pdfs p ON t.pdfName = p.pdfName LEFT JOIN projects pr ON p.application_id = pr.application_id LEFT JOIN pdfsconsultants pc ON p.pdfName = pc.pdfName LEFT JOIN sub_vecs s ON ip.tableId = s.tableId AND ip.rowIndex = s.rowIndex AND ip.rowCounter = s.rowCounter LEFT JOIN word2vec w ON ip.tableId = w.tableId AND ip.rowIndex = w.rowIndex AND ip.rowCounter = s.rowCounter;"
with engine.connect() as conn:
    issues_parsed_df = pd.read_sql(query, conn)
issues_parsed_df.fillna('', inplace = True)
issues_parsed_df = issues_parsed_df.astype(str)
issues_parsed_df = issues_parsed_df.groupby(['tableId', 'rowIndex', 'rowCounter'], as_index = False, sort = False).agg(lambda x: ', '.join(set(x))) # https://stackoverflow.com/questions/50357837/aggregating-string-columns-using-pandas-groupby
issues_parsed_df.to_csv('test.csv')
len(issues_parsed_df)

9724

### Data of Issues with no Sub VECs for SystemOps

In [ ]:
query = "SELECT distinct i.tableId, i.rowIndex , i.vec_pri, i.vec_sec, i.issue_pri, i.issue_sec, w.word2vec_vec FROM issues i LEFT JOIN issues_parsed ip ON i.tableId = ip.tableId AND i.rowIndex = ip.rowIndex LEFT JOIN sub_vecs s ON i.tableId = s.tableId AND i.rowIndex = s.rowIndex LEFT JOIN word2vec w ON i.tableId = w.tableId AND i.rowIndex = w.rowIndex WHERE s.sub_vec IS NULL AND i.status is not null;"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df['word2vec_vec'].fillna('', inplace = True)
aggr_vec_df = df.groupby(['tableId','rowIndex'], as_index = False).agg({'word2vec_vec': ', '.join}) #https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
final_issues_df = pd.merge(aggr_vec_df, df, how = 'left', on = ['tableId', 'rowIndex'])
final_issues_df = final_issues_df.drop_duplicates(subset = ['tableId', 'rowIndex'], keep = 'last').reset_index(drop = True)
del final_issues_df['word2vec_vec_y']
final_issues_df.rename(columns={'word2vec_vec_x':'VECs'}, inplace=True)
print(len(final_issues_df))
#final_issues_df.to_csv('issues.csv', encoding = 'utf-8-sig')

In [ ]:
query = "SELECT distinct ip.tableId, ip.rowIndex , ip.rowCounter, ip.issue_parsed, w.word2vec_vec FROM issues_parsed ip LEFT JOIN sub_vecs s ON ip.tableId = s.tableId AND ip.rowIndex = s.rowIndex AND ip.rowCounter = s.rowCounter LEFT JOIN word2vec w ON ip.tableId = w.tableId AND ip.rowIndex = w.rowIndex AND ip.rowCounter = w.rowCounter WHERE s.sub_vec IS NULL;"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df['word2vec_vec'].fillna('', inplace = True)
aggr_vec_df = df.groupby(['tableId','rowIndex', 'rowCounter'], as_index = False).agg({'word2vec_vec': ', '.join}) #https://stackoverflow.com/questions/27298178/concatenate-strings-from-several-rows-using-pandas-groupby
final_issues_df = pd.merge(aggr_vec_df, df, how = 'left', on = ['tableId', 'rowIndex', 'rowCounter'])
final_issues_df = final_issues_df.drop_duplicates(subset = ['tableId', 'rowIndex', 'rowCounter'], keep = 'last').reset_index(drop = True)
del final_issues_df['word2vec_vec_y']
final_issues_df.rename(columns={'word2vec_vec_x':'VECs'}, inplace=True)
print(len(final_issues_df))
final_issues_df.to_csv('issues_parsed.csv', encoding = 'utf-8-sig')

### Creating csv files

In [145]:
query = "SELECT tableId, rowIndex , content FROM issues;"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)
df

,tableId,rowIndex,content
0,0114daa6-c048-4081-b4cf-7a9d7461fa44,1,"[[""Land Use"", ""Condition/ Issues Noted (Rating..."
1,0114daa6-c048-4081-b4cf-7a9d7461fa44,2,"[[""Land Use"", ""Condition/ Issues Noted (Rating..."
2,0114daa6-c048-4081-b4cf-7a9d7461fa44,3,"[[""Land Use"", ""Condition/ Issues Noted (Rating..."
3,0114daa6-c048-4081-b4cf-7a9d7461fa44,4,"[[""Land Use"", ""Condition/ Issues Noted (Rating..."
4,0114daa6-c048-4081-b4cf-7a9d7461fa44,5,"[[""Land Use"", ""Condition/ Issues Noted (Rating..."
...,...,...,...
5185,ffa93da2-0fa1-447a-9930-bb7be8c57ac2,6,"[[""Parcel Location"", ""March 2012"", ""December 2..."
5186,ffa93da2-0fa1-447a-9930-bb7be8c57ac2,7,"[[""Parcel Location"", ""March 2012"", ""December 2..."
5187,ffa93da2-0fa1-447a-9930-bb7be8c57ac2,8,"[[""Parcel Location"", ""March 2012"", ""December 2..."
5188,ffa93da2-0fa1-447a-9930-bb7be8c57ac2,9,"[[""Parcel Location"", ""March 2012"", ""December 2..."


In [148]:
for row in df.itertuples():
    table = json.loads(row.content)
    data = pd.DataFrame(table)
    data.to_csv(r"G:\Post Construction\csvs\\" + row.tableId + '_' + str(row.rowIndex) + '.csv', encoding = 'utf-8-sig', index = False, header = None)

### Prepare data for Visualization

In [182]:
query = "SELECT i.tableId, i.rowIndex, ip.rowCounter, i.land_use_standardized, s.sub_vec, i.status, ip.status_issue_parsed, w.vec FROM issues i LEFT JOIN issues_parsed ip ON i.tableId = ip.tableId AND i.rowIndex = ip.rowIndex LEFT JOIN sub_vecs s ON i.tableId = s.tableId AND i.rowIndex = s.rowIndex AND (ip.rowCounter = s.rowCounter OR ip.rowCounter IS NULL) LEFT JOIN word2vec w ON i.tableId = w.tableId AND i.rowIndex = w.rowIndex AND (ip.rowCounter = w.rowCounter OR ip.rowCounter IS NULL);"
with engine.connect() as conn:
    df = pd.read_sql(query, conn)

In [186]:
vecs = ['air', 'fish', 'heritage', 'housekeeping', 'navigation', 'species', 'wildlife', 'water']
df['vec'].fillna('', inplace = True)
for idx, row in df.iterrows():
    for vec in vecs:
        if vec in row.vec:
            df.at[idx, 'sub_vec'] = ''

In [189]:
filter_df = df[~(df['vec'].str.contains('vegetation') & ~df['sub_vec'].isin(["Weed", "Vegetation Establishment", "Rare Plant", "Invasive Plant"])) & 
~(df['vec'].str.contains('physical') & ~df['sub_vec'].isin(["Erosion", "Coarse Fragment", "Subsidence"])) & 
~(df['vec'].str.contains('soil') & ~df['sub_vec'].isin(["Compaction", "Topsoil Loss"])) &
~(df['vec'].str.contains('wetlands') & ~df['sub_vec'].isin(["Riparian Vegetation Re-establishment"]))]

In [190]:
filter_df.to_csv('test.csv')

In [ ]:
df1 = pd.read_csv('issue_parsed_clean.csv', encoding = 'cp1252')
print(df1.columns, len(df1))

In [ ]:
df2 = pd.read_csv('v_new_parsed_issues.csv', encoding = 'utf-8-sig')
df2 = df2.loc[~df2['tableId'].isin(["3e9e6cdb-f812-4832-b69c-b8ec0396d585", "44a33e5f-d99e-48ef-ad56-bbb516ec8796", "bfafbfd0-8bb5-4283-8f5e-dd7cbcec480c"])]
print(df2.columns, len(df2))

In [ ]:
total_len = len(df1) + len(df2)
total_len

In [ ]:
df = pd.concat([df1, df2])

In [ ]:
df.to_csv('issue_parsed_clean1.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
def clean_dataframe(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    pattern = '|'.join(['<s>', '</s>',])
    dataframe[column_name] = dataframe[column_name].str.replace(pattern, '')
    dataframe[column_name] = dataframe[column_name].str.replace('\(s\)', 's') #reference: https://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandashttps://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandas
    return dataframe    

In [ ]:
query = "SELECT * FROM issues WHERE tableId = '3e1c53b4-5c01-46e2-bd72-5a338b5852f9';"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)
    #clean_dataframe(df, 'table_content')
dic = {}

for row in df.itertuples():
    # converting JSON string to a list of lists of strings
    table = json.loads(row.table_content)
    headers = table[0]  # column headers  
    for header in headers:
        #header = lemmawordnet(header)
        #header = clean_header(header)
        #header = lemmaspacy(header)
        #header = re.sub(r'\(.*?\)', lambda x: ''.join(x.group(0).split()), header) # removing whitespace between parentheses (reference: https://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-brackethttps://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-bracket)
        header = " ".join(header.split())
        if header in dic:
            dic[header] += 1
        else:
            dic[header] = 1

my_list = [(header, count) for header, count in dic.items()]  # Converting to list
my_list.sort(key=lambda tup: tup[1], reverse=True)  # sorting the list

total_headers = 0
print(f"COUNT\tHEADER")
for item in my_list:
    total_headers += item[1]
    print(f"{item[1]}\t{item[0]}")
print()
print(f'Total headers: {total_headers}; unique headers: {len(my_list)}')

In [ ]:
df3 = pd.DataFrame(my_list)
df3.to_csv('headers4.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    print(type(table))
    lst_element1 = "SPREAD"
    lst_element2 = "QUARTER SECTION"
    if lst_element1 in table[0][0] and lst_element2 in table[0][1]:
        print(row.tableId)